In [1]:
!pip install --quiet sentence-transformers faiss-cpu numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 93.5 MB/s eta 0:00:00


In [2]:
# Cell 1
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import math
import json
import re

In [3]:
# Cell 2
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Loaded embedder:", embed_model.__class__.__name__)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded embedder: SentenceTransformer


In [10]:
# Cell 3
import nltk
# If first time run in Colab uncomment these:
!pip install --quiet nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize

def text_to_chunks_by_sentence(text, max_words=300, overlap_words=60):
    # split into sentences then group into chunks of ~max_words
    sents = sent_tokenize(text)
    chunks = []
    cur = []
    cur_len = 0
    for s in sents:
        w = s.split()
        if cur_len + len(w) > max_words and cur:
            chunks.append(" ".join(cur).strip())
            # keep overlap: take last overlap_words from cur into next
            if overlap_words > 0:
                tail = " ".join(" ".join(cur).split()[-overlap_words:])
                cur = tail.split()
                cur_len = len(cur)
            else:
                cur = []
                cur_len = 0
        cur.extend(w)
        cur_len += len(w)
    if cur:
        chunks.append(" ".join(cur).strip())
    return chunks


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
# Cell 4
sample = ("Machine learning models sometimes overfit when trained too long on small data. "
          "Cross-validation helps estimate generalization. Regularization penalizes complexity. ")*40

chunks = text_to_chunks_by_sentence(sample, max_words=200, overlap_words=40)
print("Num chunks:", len(chunks))
print("Chunk[0] preview:", chunks[0][:200])


Num chunks: 5
Chunk[0] preview: Machine learning models sometimes overfit when trained too long on small data. Cross-validation helps estimate generalization. Regularization penalizes complexity. Machine learning models sometimes ov


In [12]:
# Cell 5
def build_faiss_index(chunks, embedder, save_index_path=None):
    # embed in batches
    batch_size = 32
    vecs = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        em = embedder.encode(batch, convert_to_numpy=True, show_progress_bar=False)
        vecs.append(em)
    vecs = np.vstack(vecs).astype('float32')
    faiss.normalize_L2(vecs)  # normalize for cosine via inner product
    d = vecs.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(vecs)
    if save_index_path:
        faiss.write_index(index, save_index_path)
    return index, vecs

# Build on the sample chunks for demo
index, vecs = build_faiss_index(chunks, embed_model)
print("Index size:", index.ntotal, "dim:", vecs.shape[1])


Index size: 5 dim: 384


In [13]:
# Cell 6
def retrieve(index, embedder, chunks, query, top_k=5):
    qvec = embedder.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(qvec)
    k = min(top_k, index.ntotal)
    D, I = index.search(qvec, k)
    results = []
    for score, idx in zip(D[0], I[0]):
        results.append((chunks[idx], float(score), int(idx)))
    return results

# test
q = "How to reduce overfitting?"
res = retrieve(index, embed_model, chunks, q, top_k=3)
for r in res:
    print("score", r[1], "idx", r[2], "preview:", r[0][:120])


score 0.3896132707595825 idx 0 preview: Machine learning models sometimes overfit when trained too long on small data. Cross-validation helps estimate generaliz
score 0.38632744550704956 idx 4 preview: penalizes complexity. Machine learning models sometimes overfit when trained too long on small data. Cross-validation he
score 0.3844781517982483 idx 2 preview: penalizes complexity. Machine learning models sometimes overfit when trained too long on small data. Cross-validation he


In [14]:
# Cell 7
def mmr(query_embedding, doc_embeddings, docs, top_k=5, lambda_param=0.6):
    """
    query_embedding: np.array (dim,)
    doc_embeddings: np.array (N, dim) normalized
    docs: list of docs (N)
    returns top_k indices by MMR
    lambda_param: trade-off between relevance and diversity (0..1)
    """
    # cosine similarities via dot (both normalized)
    sim_to_query = np.dot(doc_embeddings, query_embedding)
    selected = []
    not_selected = list(range(len(docs)))
    # first select highest relevance
    first = int(np.argmax(sim_to_query))
    selected.append(first)
    not_selected.remove(first)
    while len(selected) < min(top_k, len(docs)):
        mmr_score = []
        for idx in not_selected:
            relevance = sim_to_query[idx]
            diversity = max(np.dot(doc_embeddings[idx], doc_embeddings[s]) for s in selected)
            score = lambda_param * relevance - (1-lambda_param) * diversity
            mmr_score.append((score, idx))
        mmr_score.sort(reverse=True)
        chosen = mmr_score[0][1]
        selected.append(chosen)
        not_selected.remove(chosen)
    return selected

# test MMR on retrieved set
qvec = embed_model.encode([q], convert_to_numpy=True).astype('float32')
faiss.normalize_L2(qvec)
doc_embs = vecs  # all embeddings
top_candidates = retrieve(index, embed_model, chunks, q, top_k=10)
cand_indices = [c[2] for c in top_candidates]
cand_embs = doc_embs[cand_indices]
sel = mmr(qvec[0], cand_embs, [chunks[i] for i in cand_indices], top_k=4, lambda_param=0.7)
print("Selected candidate indices (local):", sel)
for s in sel:
    print("->", cand_indices[s], chunks[cand_indices[s]][:120])


Selected candidate indices (local): [0, 1, 4, 3]
-> 0 Machine learning models sometimes overfit when trained too long on small data. Cross-validation helps estimate generaliz
-> 4 penalizes complexity. Machine learning models sometimes overfit when trained too long on small data. Cross-validation he
-> 1 penalizes complexity. Machine learning models sometimes overfit when trained too long on small data. Cross-validation he
-> 2 penalizes complexity. Machine learning models sometimes overfit when trained too long on small data. Cross-validation he


In [15]:
# Cell 8
def build_rag_prompt_from_indices(question, chunks, indices, cite=True, max_context_chars=2000):
    context = ""
    for idx in indices:
        context += f"[chunk {idx}]\n" + chunks[idx].strip() + "\n\n---\n\n"
        if len(context) > max_context_chars:
            break
    prompt = f"""You are an expert HR manager. Use the following context from documents to answer the question.
If the answer is not in the context, say 'I don't know' or answer conservatively.

Context:
{context}

Question: {question}

Answer concisely and cite chunk indices when referencing context.
"""
    return prompt

# Example: retrieve many then MMR pick
candidates = retrieve(index, embed_model, chunks, q, top_k=12)
cand_indices = [c[2] for c in candidates]
cand_embs = vecs[cand_indices]
qvec = embed_model.encode([q], convert_to_numpy=True).astype('float32')
faiss.normalize_L2(qvec)
selected_local_positions = mmr(qvec[0], cand_embs, [chunks[i] for i in cand_indices], top_k=4, lambda_param=0.6)
selected_indices = [cand_indices[pos] for pos in selected_local_positions]
print("Selected indices:", selected_indices)

prompt = build_rag_prompt_from_indices(q, chunks, selected_indices)
print(prompt[:600])
# send prompt to your LLM (use your existing generate_text/generate_from_llm)


Selected indices: [0, 4, 1, 2]
You are an expert HR manager. Use the following context from documents to answer the question. 
If the answer is not in the context, say 'I don't know' or answer conservatively.

Context:
[chunk 0]
Machine learning models sometimes overfit when trained too long on small data. Cross-validation helps estimate generalization. Regularization penalizes complexity. Machine learning models sometimes overfit when trained too long on small data. Cross-validation helps estimate generalization. Regularization penalizes complexity. Machine learning models sometimes overfit when trained too long on small d
